# Задание 1

Имплементируйте алгоритм Леска (описание есть в семинаре) и оцените качество его работы на датасете `data/corpus_wsd_50k.txt`

В качестве метрики близости вы должны попробовать два подхода:

1) Jaccard score на множествах слов (определений и контекста)
2) Cosine distance на эмбедингах sentence_transformers

В качестве метрики используйте accuracy (% правильных ответов). Предсказывайте только многозначные слова в датасете

Контекст вы можете определить самостоятельно (окно вокруг целевого слова или все предложение). Также можете поэкспериментировать с предобработкой для обоих методов.

In [3]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Home\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
from nltk.corpus import wordnet as wn

In [31]:
corpus_wsd = []
corpus = open('data/corpus_wsd_50k.txt').read().split('\n\n')
for sent in corpus:
    corpus_wsd.append([s.split('\t') for s in sent.split('\n')])

In [ ]:
corpus_wsd[:50]

In [ ]:
!python -m pip install torch torchvision torchaudio
!python -m pip install sentence_transformers transformers accelerate -U

In [10]:
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity
from sentence_transformers import SentenceTransformer

c:\Users\Home\scoop\apps\python\current\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [31]:
import numpy as np

Функция измерения близости по Jaccard Score

In [4]:
def jaccard_score(sent: str, cont: str) -> int:
    intersection = (set(sent.split()) & set(cont.split()))
    union = (set(sent.split()) | set(cont.split()))
    return len(intersection) / len(union)

Функция измерения близости по Cosine

In [11]:
# embedding model
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
embed = model.encode

Здесь я использую косинусное расстояние

In [26]:
def cosine_score(sent: str, cont: str) -> int:
    sent_emb = embed(sent)
    cont_emb = embed(cont)
    return cosine_distances(cont_emb.reshape(1, -1), sent_emb.reshape(1, -1))[0][0]

Здесь я использую косинусную близость + нахожу номер элемента с наибольшей близостью

In [27]:
def cosine_score_2(sent: str, cont: list) -> int:
    sent_emb = embed(sent)
    cont_emb =[]
    for definition in cont:
        cont_emb.append(embed(definition))
    scores = cosine_similarity(sent_emb.reshape(1, -1), cont_emb)
    max_score = max(scores[0])
    index_of_max = scores[0].tolist().index(max_score)
    return [max_score, index_of_max]
    
#Возврщает список, с указанием номера лучшего определения из списка определений ворднета

In [ ]:
cosine_score_2('bear',['apple','bear claws forest strong', 'toy'])

[0.533312, 1]

## Алгоритм Леска с Jaccard Score

In [84]:
def lesk_jaccard(word: str, sentence: str, compare_how: jaccard_score) -> int:

    bestsense = 0
    maxoverlap = 0
    word=wn.morphy(word) if wn.morphy(word) is not None else word
    synsets = wn.synsets(word)
    
    for syns in synsets:
        overlap = compare_how(sentence, syns.definition())
                
        if overlap > maxoverlap:
            maxoverlap = overlap
            bestsense = syns
    return bestsense

In [86]:
# Проверка работы функции
lesk_jaccard('bear','that huge mammal is living in a forest. it has massive strong claws',compare_how=jaccard_score)

Synset('bear.n.01')

In [75]:
true = 0
total = 0

for sent in corpus_wsd[0:1000]:
    for word in sent:
        code, lemma, word = word
        if not code:
            continue
        total+=1
        synset_actual = wn.lemma_from_key(code).synset()
        context = " ".join([s[2] for s in sent])
        synset_chosen = lesk_jaccard(word, context, compare_how=jaccard_score)
        
        try:
            if synset_actual==synset_chosen:
                true+=1
        except:
            pass
            
print('accuracy: %f' % (round((true/total) * 100, 1)))    

percentage of correct: 20.100000


Измеряя близость с помощью метрики Jaccard, мы получили Accuracy = 20.1%

## Алгоритм Леска с косинусом

Сначала попробуем косинусное расстояние

In [28]:
def lesk_cosine(word: str, sentence: str, compare_how: cosine_score) -> int:

    bestsense = 0
    minoverlap = 1
    word=wn.morphy(word) if wn.morphy(word) is not None else word
    synsets = wn.synsets(word)
    
    for syns in synsets:
        overlap = compare_how(sentence, syns.definition())
                
        if overlap < minoverlap:
            minoverlap = overlap
            bestsense = syns
    return bestsense

In [70]:
true = 0
total = 0

for sent in corpus_wsd[0:10]:
    for word in sent:
        code, lemma, word = word
        if not code:
            continue
        total+=1
        synset_actual = wn.lemma_from_key(code).synset()
        context = " ".join([s[2] for s in sent])
        synset_chosen = lesk_cosine(word, context, compare_how=cosine_score)
        
        try:
            if synset_actual==synset_chosen:
                true+=1
        except:
            pass
            
print('accuracy: %f' % (round((true/total) * 100, 1)))

percentage of correct: 37.300000


Теперь попробуем с косинусной близостью. Предполагалось, что такой подход уменьшит длительность выполнения операции, но нет :(
Все равно приходится ждать

In [29]:
def lesk_cosine_2(word: str, sentence: str, compare_how: cosine_score_2) -> int:

    bestsense = 0
    
    word=wn.morphy(word) if wn.morphy(word) is not None else word
    synsets = wn.synsets(word)
    list_of_definitions = [definition.definition() for definition in synsets]
    bestsense = compare_how(sentence, list_of_definitions)
    num_of_best = bestsense[1]
    bestsyn = synsets[num_of_best]

    return bestsyn

In [63]:
# Проверка работы функции
lesk_cosine_2('bear','that huge mammal is living in a forest. it has massive strong claws', compare_how=cosine_score_2)


Synset('bear.n.01')

In [40]:
true = 0
total = 0

for sent in corpus_wsd[0:10]:
    for word in sent:
        code, lemma, word = word
        if not code:
            continue
        total+=1
        synset_actual = wn.lemma_from_key(code).synset()
        context = " ".join([s[2] for s in sent])
        synset_chosen = lesk_cosine_2(word, context, compare_how=cosine_score_2)
        
        try:
            if synset_actual==synset_chosen:
                true+=1
        except:
            pass
            
print('accuracy: %f' % (round((true/total) * 100, 1)))

accuracy: 38.800000


Измеряя близость с помощью метрики Косинусного расстояния (и близости), мы получили Accuracy = 39.2%
Однако, проверка выполняется долго (15 минут / 50 предложений из корпуса) 

# Задание 2
Попробуйте разные алгоритмы кластеризации на датасете - `https://github.com/nlpub/russe-wsi-kit/blob/initial/data/main/wiki-wiki/train.csv`

Используйте код из семинара как основу. Используйте ARI как метрику качества.

Попробуйте все 4 алгоритма кластеризации, про которые говорилось на семинаре. Для каждого из алгоритмов попробуйте настраивать гиперпараметры (посмотрите их в документации). Прогоните как минимум 5 экспериментов (не обязательно успешных) с разными параметрами на каждый алгоритме кластеризации и оцените: качество кластеризации, скорость работы, интуитивность параметров.

Помимо этого также выберите 1 дополнительный алгоритм кластеризации отсюда - https://scikit-learn.org/stable/modules/clustering.html , опишите своими словами принцип его работы  и проделайте аналогичные эксперименты. 

In [71]:
!pip install pandas

  Obtaining dependency information for pandas from https://files.pythonhosted.org/packages/2d/5e/9213ea10ac473e2437dc2cb17323ddc0999997e2713d6a0b683b10773994/pandas-2.1.1-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for pytz>=2020.1 from https://files.pythonhosted.org/packages/32/4d/aaf7eff5deb402fd9a24a1449a8119f00d74ae9c2efa79f8ef9994261fc2/pytz-2023.3.post1-py2.py3-none-any.whl.metadata
     ---------------------------------------- 0.0/341.8 kB ? eta -:--:--
     --- --------------------------------- 30.7/341.8 kB 640.0 kB/s eta 0:00:01
     ---------- ---------------------------- 92.2/341.8 kB 1.1 MB/s eta 0:00:01
     --------------------- ---------------- 194.6/341.8 kB 1.7 MB/s eta 0:00:01
     -------------------------- ----------- 235.5/341.8 kB 1.4 MB/s eta 0:00:01
     ----------------------------------- -- 317.4/341.8 kB 1.5 MB/s eta 0:00:01
     -------------------------------------  337.9/341.8 kB 1.3 MB/s eta 0:00:01
     ------------------------

In [1]:
import pandas as pd

In [23]:
from sklearn.cluster import KMeans, DBSCAN, AffinityPropagation, Birch
import numpy as np
from sklearn.metrics import adjusted_rand_score

In [5]:
df = pd.read_csv('https://raw.githubusercontent.com/nlpub/russe-wsi-kit/initial/data/main/wiki-wiki/train.csv', sep='\t')

In [6]:
grouped_df = df.groupby('word')[['word', 'context', 'gold_sense_id']]

In [7]:
for key, _ in grouped_df:
    print(grouped_df.get_group(key), "\n\n")
    break

    word                                            context  gold_sense_id
383  бор  бор ( элемент ) бор — элемент тринадцатой груп...              1
384  бор  бор - углерод - кремний семейство сплавов на о...              1
385  бор  с большим выделением теплоты , образуется окси...              1
386  бор  это объясняется прежде всего тем , что у компл...              1
387  бор  совсем малочисленна . элементарный бор в приро...              1
388  бор  действующем при месторождении горно - химическ...              1
389  бор  b c ) . при нагревании в атмосфере кислорода и...              1
390  бор  собственных минералов бора в чужих минералах о...              1
391  бор  бор - углерод - кремний семейство сплавов на о...              1
392  бор  с другими галогенами с образованием тригалоген...              1
393  бор  и в стабильны и входят в состав природного бор...              1
394  бор  номером . обозначается символом b ( ) . в своб...              1
395  бор  и взаимные пере

Эксперимент 1  
Метод: k-means  
Параметры: n_clusters = 2, n_init='auto', algorithm='elkan'  
Время исполнения: 10 минут 47 сек  
ARI = 0.050794839682544106  

Эксперимент 2  
Метод: k-means  
Параметры: n_clusters = 5, max_iter = 100  
Время исполнения: 10 минут 21 сек  
ARI = 0.038048835085311684  

Эксперимент 3  
Метод: AffinityPropagation  
Параметры: damping=0.5, max_iter=100  
Время исполнения: 10 минут  
ARI = 0.042363774919161074  

Эксперимент 4  
Метод: AffinityPropagation   
Параметры: damping=0.7, verbose = True  
Время исполнения: 10 минут 13 сек  
ARI = 0.04154515818974152  

Эксперимент 5  
Метод: DBSCAN  
Параметры: min_samples=5, eps=0.5, algorithm='brute'  
Время исполнения: 10 минут 19 сек  
ARI = -0.011271692824715207  

Эксперимент 6  
Метод: DBSCAN  
Параметры: min_samples=4, eps=0.1, algorithm='ball_tree'  
Время исполнения: 10 минут 13 сек  
ARI = -0.00867271551782085  

Пробую метод кластеризации Birch  
BIRCH это метод разделения данных на группы с использованием специального дерева, которое позволит нам более эффективно  организовывать точки в кластеры в иерархии

Эксперимент 7
Метод: Birch  
Параметры: threshold=0.5, branching_factor=50, n_clusters=3, compute_labels=True, copy=True 
Время исполнения: 13 минут 30 сек
ARI = 0.018957481003908684 

In [24]:
ARI = []

for key, _ in grouped_df:
    # вытаскиваем контексты
    texts = grouped_df.get_group(key)['context'].values

    # создаем пустую матрицу для векторов 
    X = np.zeros((len(texts), 768))

    # переводим тексты в векторы и кладем в матрицу
    for i, text in enumerate(texts):
        X[i] = embed(text)

    # выбираем один из алгоритмов
    
    #cluster = KMeans(n_clusters = 2, n_init='auto', algorithm='elkan')
    #cluster = KMeans(n_clusters = 5, max_iter = 100)
   
    #cluster = AffinityPropagation(damping=0.5, max_iter=100)
    #cluster = AffinityPropagation(damping=0.7, verbose = True)

    #cluster = DBSCAN(min_samples=5, eps=0.5, algorithm='brute')
    #cluster = DBSCAN(min_samples=4, eps=0.1, algorithm='ball_tree')

    cluster = Birch(threshold=0.5, branching_factor=50, n_clusters=3, compute_labels=True, copy=True)

    cluster.fit(X)
    labels = np.array(cluster.labels_)+1 

    # расчитываем метрику для отдельного слова
    ARI.append(adjusted_rand_score(grouped_df.get_group(key)['gold_sense_id'], labels))
    
print(np.mean(ARI)) # усредненная метрика

c:\Users\Home\scoop\apps\python\current\Lib\site-packages\sklearn\cluster\_birch.py:725: ConvergenceWarning: Number of subclusters found (2) by BIRCH is less than (3). Decrease the threshold.
  warnings.warn(


0.018957481003908684
